# Visual Question Answering (VQA)
This notebook implements a multi-modal architecture for VQA based on the DAQUAR dataset introduced here: [DAQUAR dataset](https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/research/vision-and-language/visual-turing-challenge/)

The objective of this project is to merge two modalities: Images + Text and build an AI that can answer questions about images.

## Dataset

The processed version of the dataset used in this project can be downloaded from Kaggle [here](https://www.kaggle.com/datasets/tezansahu/processed-daquar-dataset/data). Also contains the original dataset files.

### Original dataset
The original dataset has 3 files:
- `all_qa_pairs.txt`
- `train_images_list.txt`
- `test_images_list.txt`

### Processed dataset

The processed version dataset contains a processed version of the full DAQUAR dataset, with the following descriptions:
- `data.csv`: Processed dataset after normalizing all the questions and conversting the data into a tabular format {question, answer, image_id}
- `data_train.csv`: Training data from `train_images_list.txt`
- `data_eval.csv`: Testing data from `test_images_list.txt`
- `answer_space.txt`: List of all possible answers extracted from `all_qa_pairs.txt`